# **Regresión Lineal Múltiple**

In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

In [5]:
# Generar un dataset genérico
np.random.seed(42)
n_samples = 100
X_numerical = np.random.rand(n_samples, 2)  # Dos variables numéricas
X_categorical = np.random.choice(['A', 'B', 'C'], size=n_samples).reshape(-1, 1)
y = 3 * X_numerical[:, 0] + 5 * X_numerical[:, 1] + np.random.randn(n_samples) * 0.5

df = pd.DataFrame(np.hstack([X_numerical, X_categorical]), columns=['feature1', 'feature2', 'category'])
df['target'] = y

df.head()

,feature1,feature2,category,target
0,0.3745401188473625,0.9507143064099162,C,5.101475
1,0.7319939418114051,0.5986584841970366,A,4.934028
2,0.15601864044243652,0.15599452033620265,B,0.881553
3,0.05808361216819946,0.8661761457749352,B,4.602805
4,0.6011150117432088,0.7080725777960455,C,5.065735


In [7]:
# Normalización
scaler = StandardScaler()
df[['feature1', 'feature2']] = scaler.fit_transform(df[['feature1', 'feature2']])
df.head()

,feature1,feature2,category,target
0,-0.350389,1.633029,C,5.101475
1,0.825606,0.393304,A,4.934028
2,-1.069308,-1.165488,B,0.881553
3,-1.391507,1.335338,B,4.602805
4,0.395025,0.778593,C,5.065735


In [9]:
# One-Hot Encoding
encoder = OneHotEncoder(drop='first')
encoded_cats = encoder.fit_transform(df[['category']]).toarray()
encoded_cols = encoder.get_feature_names_out(['category'])
df_encoded = pd.DataFrame(encoded_cats, columns=encoded_cols)
df_encoded.head()

,category_B,category_C
0,0.0,1.0
1,0.0,0.0
2,1.0,0.0
3,1.0,0.0
4,0.0,1.0


In [15]:
# Unir los datos normalizados y codificados
X = np.hstack([df[['feature1', 'feature2']].values, df_encoded.values])
X = np.c_[np.ones(X.shape[0]), X]  # Agregar columna de 1s para el término independiente
y = df['target'].values.reshape(-1, 1)

In [14]:
# Partición en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Sección 1: Implementación desde cero

In [16]:
def linear_regression(X, y):
    # Cálculo del vector de coeficientes usando la fórmula cerrada
    beta = np.linalg.inv(X.T @ X) @ X.T @ y
    return beta

# Entrenamiento del modelo
beta = linear_regression(X_train, y_train)
print("Coeficientes del modelo desde cero:", beta.flatten())

# Evaluación del modelo en el conjunto de prueba
y_pred_test = X_test @ beta
mse_test = np.mean((y_test - y_pred_test) ** 2)
print("Error cuadrático medio en el conjunto de prueba (desde cero):", mse_test)

Coeficientes del modelo desde cero: [ 3.94602924  0.89391038  1.46092924 -0.14399343 -0.1028087 ]
Error cuadrático medio en el conjunto de prueba (desde cero): 0.34860108146495117


# Sección 2: Implementación con scikit-learn

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Crear el modelo
model = LinearRegression()
model.fit(X_train[:, 1:], y_train)  # Excluimos la columna de 1s para que scikit-learn gestione el intercepto

# Evaluación del modelo en el conjunto de prueba
y_pred_test_sklearn = model.predict(X_test[:, 1:])
mse_test_sklearn = mean_squared_error(y_test, y_pred_test_sklearn)

# Evaluciones y Comparaciones

In [18]:


print("Coeficientes del modelo con scikit-learn:", model.coef_.flatten())
print("Intercepto del modelo con scikit-learn:", model.intercept_[0])
print("Error cuadrático medio en el conjunto de prueba (scikit-learn):", mse_test_sklearn)

Coeficientes del modelo con scikit-learn: [ 0.89391038  1.46092924 -0.14399343 -0.1028087 ]
Intercepto del modelo con scikit-learn: 3.9460292407033757
Error cuadrático medio en el conjunto de prueba (scikit-learn): 0.34860108146495233


# **Un ejemplo práctico**

In [30]:
from sklearn.datasets import load_wine

# Cargar el dataset Wine
data = load_wine()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

df.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


### Explicación del Dataset `Wine`

El dataset `Wine` es un conjunto de datos clásico disponible en `scikit-learn`, que contiene información química de diferentes muestras de vino. Este dataset se utiliza comúnmente para problemas de clasificación, pero aquí lo estamos usando para una regresión lineal múltiple.

- **Cantidad de muestras:** 178 vinos.
- **Cantidad de características:** 13 características numéricas.
- **Clases de vinos:** 3 clases (0, 1, 2), que representan tres cultivares diferentes de vino.

#### **Características:**

Las características del dataset son mediciones químicas de cada muestra de vino. A continuación, se enumeran:

1. `alcohol`: Porcentaje de alcohol.
2. `malic_acid`: Ácido málico.
3. `ash`: Ceniza.
4. `alcalinity_of_ash`: Alcalinidad de la ceniza.
5. `magnesium`: Magnesio.
6. `total_phenols`: Fenoles totales.
7. `flavanoids`: Flavonoides.
8. `nonflavanoid_phenols`: Fenoles no flavonoides.
9. `proanthocyanins`: Proantocianinas.
10. `color_intensity`: Intensidad del color.
11. `hue`: Tono del color.
12. `od280/od315_of_diluted_wines`: Relación OD280/OD315 de vinos diluidos.
13. `proline`: Prolina.

#### **Variable objetivo (`target`):**

- Esta variable es categórica y representa la clase del vino (`0`, `1` o `2`).

En el código, hemos tratado la variable `target` como categórica y la hemos codificado mediante **One-Hot Encoding**, pero también podría tratarse como una variable numérica si se quisiera realizar una regresión.


In [32]:
# Seleccionar variables numéricas y categóricas
numerical_features = df.columns[:-1]
categorical_features = ['target']
print(categorical_features)

['target']


In [33]:
# Normalización
scaler = StandardScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])
df[numerical_features].head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,1.518613,-0.562250,0.232053,-1.169593,1.913905,0.808997,1.034819,-0.659563,1.224884,0.251717,0.362177,1.847920,1.013009
1,0.246290,-0.499413,-0.827996,-2.490847,0.018145,0.568648,0.733629,-0.820719,-0.544721,-0.293321,0.406051,1.113449,0.965242
2,0.196879,0.021231,1.109334,-0.268738,0.088358,0.808997,1.215533,-0.498407,2.135968,0.269020,0.318304,0.788587,1.395148
3,1.691550,-0.346811,0.487926,-0.809251,0.930918,2.491446,1.466525,-0.981875,1.032155,1.186068,-0.427544,1.184071,2.334574
4,0.295700,0.227694,1.840403,0.451946,1.281985,0.808997,0.663351,0.226796,0.401404,-0.319276,0.362177,0.449601,-0.037874


In [35]:
# One-Hot Encoding
encoder = OneHotEncoder(drop='first')
encoded_cats = encoder.fit_transform(df[categorical_features]).toarray()
encoded_cols = encoder.get_feature_names_out(categorical_features)
df_encoded = pd.DataFrame(encoded_cats, columns=encoded_cols)
df_encoded.head()

,target_1,target_2
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0


In [40]:
# Unir los datos normalizados y codificados
X = np.hstack([df[numerical_features].values, df_encoded.values])
X = np.c_[np.ones(X.shape[0]), X]  # Agregar columna de 1s para el término independiente
y = df['target'].values.reshape(-1, 1)

In [25]:
# Partición en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
# Sección 1: Implementación desde cero
def linear_regression(X, y):
    # Cálculo del vector de coeficientes usando la fórmula cerrada
    beta = np.linalg.inv(X.T @ X) @ X.T @ y
    return beta

In [43]:
# Entrenamiento del modelo
beta = linear_regression(X_train, y_train)
print("Coeficientes del modelo desde cero:", beta.flatten())

# Evaluación del modelo en el conjunto de prueba
y_pred_test = X_test @ beta
mse_test = np.mean((y_test - y_pred_test) ** 2)
print("Error cuadrático medio en el conjunto de prueba (desde cero):", mse_test)

Coeficientes del modelo desde cero: [-4.44089210e-15  6.38378239e-16  4.33680869e-16  6.93889390e-17
 -3.46944695e-17  6.93889390e-18 -7.14706072e-16  2.13024043e-15
  3.08780779e-16 -2.63677968e-16 -7.91033905e-16 -2.15105711e-16
  1.99840144e-15  2.63677968e-15  1.00000000e+00  2.00000000e+00]
Error cuadrático medio en el conjunto de prueba (desde cero): 1.6206771433576886e-29


In [44]:
# Sección 2: Implementación con scikit-learn
# Crear el modelo
model = LinearRegression()
model.fit(X_train[:, 1:], y_train)  # Excluimos la columna de 1s para que scikit-learn gestione el intercepto

# Evaluación del modelo en el conjunto de prueba
y_pred_test_sklearn = model.predict(X_test[:, 1:])
mse_test_sklearn = mean_squared_error(y_test, y_pred_test_sklearn)

print("Coeficientes del modelo con scikit-learn:", model.coef_.flatten())
print("Intercepto del modelo con scikit-learn:", model.intercept_[0])
print("Error cuadrático medio en el conjunto de prueba (scikit-learn):", mse_test_sklearn)

Coeficientes del modelo con scikit-learn: [-2.57249074e-16  2.77555756e-16  1.11022302e-16 -5.96744876e-16
  6.86950496e-16 -4.16333634e-16 -6.24500451e-16 -3.95516953e-16
 -9.43689571e-16 -1.24900090e-16  6.41847686e-17  3.08780779e-16
  2.08166817e-16  1.00000000e+00  2.00000000e+00]
Intercepto del modelo con scikit-learn: -1.1102230246251565e-16
Error cuadrático medio en el conjunto de prueba (scikit-learn): 1.6290065798125387e-30
